# Calculate SPG from CBS html



In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

import os
import sys

sys.path.append(os.path.normpath(r'..\phi-utils'))
print(sys.path)
# N.B. this was has not been merged to main in that repo....
from philosofool.data_sources.utils import read_yml
#os.path.exists(os.path.norm('C:Users/lenha/Repos/phi-utils'))

from fantasy_baseball_draft.spg import spgs_from_standings_html as calc_spg
from fantasy_baseball_draft.stats import hitter_fwar, pitcher_fwar
from fantasy_baseball_draft.utils import process_players, process_hitters, load_cbs_data, free_agents

In [ ]:
os.

In [ ]:
data_path = read_yml('local/config.yml')['paths']['local_data']
standings_path = os.path.join(data_path, r'standings\cbs_2021_standings.html')

spg = calc_spg(standings_path)
spg

In [ ]:
projections_path = os.path.join(data_path, r'projections')
eligibility = load_cbs_data(os.path.join(data_path, 'eligibility/eligibility.csv'))
hitters = (
    load_cbs_data(os.path.join(projections_path, 'cbs_hitters.csv'))
    .pipe(process_hitters, eligibility)
    .pipe(process_players)
    .drop(columns='player')
)
pitchers = (
    load_cbs_data(os.path.join(projections_path, 'cbs_pitchers.csv'))
    .pipe(process_players)
    .drop(columns='player')
)
free_agents(pitchers).sum()
pitchers.head()
hitters

In [ ]:

class ReplacementValuator:
    positions = ['C', '1B', '2B', '3B', 'SS', 'LF', 'CF', 'RF']
    
    def __init__(self, spg: pd.DataFrame):
        self.spg = spg


    def find_replace_values(df, depth):
        positions = ['C', '1B', '2B', '3B', 'SS', 'LF', 'CF', 'RF']
        df = df.copy().rename(columns={k: k.lower() for k in df.columns})
        df['fwar'] = hitter_fwar(df, self.spg)
        df = df.sort_values('fwar', ascending=False)
        #print(df.head())
        raw_replace = df.fwar[:depth].min()
        replace_values = {}
        for pos in self.positions:
            exp = f'{pos},|{pos}$'
            _df = df[df.eligible.str.contains(exp)]
            replace_values[pos] = max(raw_replace - _df[:depth].fwar.min(), 0)
        return replace_values

find_replace_values(hitters, 16, spg.spg, spg['median'])

In [ ]:
class FantasyValuator:
    def __init__(self, spg: pd.DataFrame):
        self.spg = spg

    def valuate_hitters(self, df, num_owned, copy=False):
        if copy:
            df = df.copy()
            self.valuate_hitters(df)
            return df
        fwar = self.hitter_fwar(df)
        replacement_level = self.replacement_level(fwar, num_owned)
        df['fwar'] = fwar - replacement_level

    def hitter_fwar(self, df) -> pd.Series:
        df = self.clean_df(df)
        return hitter_fwar(df, self.spg)

    def pitcher_fwar(self, df) -> pd.Series:
        df = df.clean_df(df)
        return pitcher_fwar(df, self.spg)

    def replacement_level(self, fwar: pd.Series, num_owned: int):
        return fwar.sort_values(ascending=False)[:num_owned].min()

    def clean_df(self, df, inplace=False) -> pd.DataFrame:
        """Clean df for processing."""
        if not inplace:
            df = df.copy()
            self.clean_df(df, inplace=True)
            return df
        df.rename({k: self.stat_concordance.normalize(k) for k in df.columns}, inplace=True)

In [ ]:
from fantasy_baseball_draft.stats import hitter_fwar, pitcher_fwar
hitter_fwar(hitters, spg['spg'], spg['median']).sort_values()[-180:]